In [503]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, \
    plot_confusion_matrix,classification_report, confusion_matrix, precision_score, \
    recall_score, f1_score


In [504]:
# Uncomment when using in Google Colab
# from google.colab import files
# uploaded = files.upload()

In [505]:
# Uncomment when using in Google Colab
# import io
# mass = pd.read_csv(io.BytesIO(uploaded['mass_mobile.csv']))


### Load cleaned data from a csv file

In [506]:
mass = pd.read_csv('../data/03_mass_no_country.csv')

In [507]:
mass.head(3)

,id,country,ccode,region,protestnumber,protesterviolence,location,protesteridentity,sources,notes,...,end_date,target,notes_clean,neg,neu,pos,compound,protest_duration,violent_response,violent_count
0,201990001,Canada,20,North America,1,0.0,national,unspecified,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...,...,1990-01-15,"[0, 0, 0, 0, 1, 0, 0]",canada s railway passenger system was finally ...,0.087,0.913,0.000,-0.8176,1,0,0
1,201990002,Canada,20,North America,2,0.0,"Montreal, Quebec",unspecified,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...,...,1990-06-25,"[0, 0, 0, 0, 1, 0, 0]",protestors were only identified as young peopl...,0.000,1.000,0.000,0.0000,1,0,0
2,201990003,Canada,20,North America,3,0.0,"Montreal, Quebec",separatist parti quebecois,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai...",...,1990-07-01,"[0, 0, 0, 0, 1, 0, 0]","the queen, after calling on canadians to remai...",0.060,0.830,0.109,0.7003,1,0,0


In [508]:
# User dataframe of almost all features.

features = mass.drop(columns=['id', 'country', 'ccode', 'region', 'location','protesteridentity', 'sources',
           'notes', 'protester_id_type', 'protest_size_category', 'start_date', 'notes_clean', 'neg', 'neu', 'pos', 'compound',
           'end_date', 'target', 'arrests', 'accomodation', 'beatings', 'crowddispersal', 'ignore', 'killings', 'shootings', 
           'partipants_number','pop_male', 'pop_female', 'violent_response', 'violent_count', 'protest_duration'])


print(f'X will be set to use {features.shape[1]} features.')

X will be set to use 40 features.


In [509]:
#country_list - this code applies to when using a different csv file that DID included a dummied column for each country
#country_column_names = list(features_1.columns[9:174])
#features = features_1.drop(country_column_names, axis=1)

In [510]:

# This code was initially used to define a small set of features just to get a small model going.
# features = ['pop_density', 'prosperity_2020','partipants_number','compound','region_Europe','region_Central America','region_Africa',
# 'region_MENA', 'region_North America', 'region_Oceania', 'region_South America']
#X = mass[features]

X = features
y = np.array(mass[['arrests', 'accomodation', 'beatings',
       'crowddispersal', 'ignore', 'killings', 'shootings']])

y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [511]:
len(X)

15179

In [512]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#### To use a Gridsearch with the MultiOutputClassifer, uncomment the cell below.

In [513]:
# set up MultiOutputClassifier
#model_grad_booster = MultiOutputClassifier(GradientBoostingClassifier())

# Set up hyperparameters
# hyperparameters_grad = dict(estimator__learning_rate=[ 0.2, 0.5], 
#                      estimator__n_estimators=[20, 50, 100],
#                      estimator__min_samples_split=[2, 4, 7, 10],
#                      estimator__max_depth=[5, 20, 30], 
#                      estimator__min_samples_leaf=[ 5, 8, 10],
#                      estimator__min_impurity_decrease=[0.2, 0.6, 0.8],
#                      estimator__max_leaf_nodes=[5, 20, 100])

# Execute Grid Search
# grid_search = GridSearchCV(model_grad_booster, hyperparameters_grad,n_jobs=-1, cv=5, verbose=True,
#                                        pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

# Fit the model
#hyperparameters_tuning = grid_search.fit(X_train, y_train)

#### Current model uses a Randomized CV Search, MultiOutputClassifier of a Random Forest

In [514]:

# set up MultiOutputClassifier
model_random_forest = MultiOutputClassifier(RandomForestClassifier())  

# Set up hyperparameters
hyperparameters_forest = dict(estimator__n_estimators=[20, 50, 100, 200, 300, 500, 1000],
                       estimator__min_samples_split=[2, 4, 7, 10],
                       estimator__max_depth=[3, 5, 10, 15, 20, 30],
                       estimator__min_samples_leaf=[1, 2, 3, 5, 8, 10],
                       estimator__min_impurity_decrease=[0, 0.2, 0.4, 0.6, 0.8],
                       estimator__max_leaf_nodes=[5, 10, 20, 30, 50, 100, 300])

# n_iter   int, default=10 - might want to increase?

# Set up Random Search criteria
randomized_search = RandomizedSearchCV(model_random_forest, hyperparameters_forest,
                                       n_jobs=-1, cv=5, verbose=1, n_iter=30,
                                       error_score='raise')

# Fit the model
hyperparameters_tuning = randomized_search.fit(X_train, y_train)

len(regr_multi_RF.estimators_)

print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

# Set model variable to be the best performing estimator
tuned_model = hyperparameters_tuning.best_estimator_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.1min finished


AttributeError: 'MultiOutputClassifier' object has no attribute 'estimators_'

#### Print best score
If no scoring method is assigned to the `RandomizedSearchCV` function, the it uses the scoring method associated with the model, which in this case is `RandomForestClassifier`.  

RandomForestClassifier's scoring function:
> Return the mean accuracy on the given test data and labels.

In [ ]:
hyperparameters_tuning.best_score_

#### Get predictions using `X_test` 

In [ ]:
y_preds = tuned_model.predict(X_test)

In [ ]:
tuned_model.score(X_train, y_train )

In [ ]:
tuned_model.score(X_test, y_test)

#### Save four rows of features so we can run a test of using the model after loading its pickle file in another notebook

In [ ]:
X_test.iloc[0:4].to_csv('../data/x_sample.csv', index=False)

#### Export tuned model to use for predictions in another notebook.

In [ ]:
# save the model to disk
model_file_name = '../models/01_multi_label_forest.pickle'
pickle.dump(  tuned_model     , open(   model_file_name, 'wb'))

---

In [ ]:
# Hat tip to this blog post:
# https://medium.com/@saugata.paul1010/a-detailed-case-study-on-multi-label-classification-with-machine-learning-algorithms-and-72031742c9aa

print("Accuracy = ", accuracy_score(y_test,y_preds))

precision = precision_score(y_test, y_preds, average='micro')
recall = recall_score(y_test, y_preds, average='micro')
f1 = f1_score(y_test, y_preds, average='micro')
 
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

#### Baseline model

In [ ]:
y_train

In [ ]:
y_df = pd.DataFrame(data=y, columns=['arrests', 'accomodation', 'beatings',
       'crowddispersal', 'ignore', 'killings', 'shootings'])
y_df[0:3]

In [ ]:
# Sum of all 1's in all columns, divided by the number of rows in the train dataset
pd.DataFrame(y_df.sum() / y_df.shape[0]).T

In [ ]:
y.shape

In [ ]:
y_df.sum(axis=1).value_counts()

### Print confusion matrix for all target variables (state responses)

The multilabel_confusion_matrix function returns an array of seven confusion matrices (on for each target variable).

Each Seaborn heatmap loads one of the matrices from the array (called `cm_array`)

In [ ]:
cm_array = multilabel_confusion_matrix(y_test, y_preds)  # this returns an array of confusion matrices
cm_array[0]

For better readability, just plot two at a time.

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
plt.subplots_adjust(wspace=0.5)

sns.heatmap(cm_array[0].astype(int), annot=True, ax = ax1, cmap='Greens', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax1.set_xlabel('Predicted State Response');
ax1.set_ylabel('Actual State Response'); 
ax1.set_title('Confusion Matrix'); 
ax1.xaxis.set_ticklabels(['No Arrests', 'Arrests']);
ax1.yaxis.set_ticklabels(['No Arrests', 'Arrests'], va='center');

sns.heatmap(cm_array[1].astype(int), annot=True, ax = ax2, cmap='Greens', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax2.set_xlabel('Predicted State Response');
ax2.set_ylabel('Actual State Response'); 
ax2.set_title('Confusion Matrix'); 
ax2.xaxis.set_ticklabels(['No Accomodation', 'Accomodation']);
ax2.yaxis.set_ticklabels(['No Accomodation', 'Accomodation'], va='center');
    

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
plt.subplots_adjust(wspace=0.5)

sns.heatmap(cm_array[2].astype(int), annot=True, ax = ax1, cmap='crest', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax1.set_xlabel('Predicted State Response');
ax1.set_ylabel('Actual State Response'); 
ax1.set_title('Confusion Matrix'); 
ax1.xaxis.set_ticklabels(['No Beatings', 'Beatings']);
ax1.yaxis.set_ticklabels(['No Beatings', 'Beatings'], va='center');

sns.heatmap(cm_array[3].astype(int), annot=True, ax = ax2, cmap='Greens', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax2.set_xlabel('Predicted State Response');
ax2.set_ylabel('Actual State Response'); 
ax2.set_title('Confusion Matrix'); 
ax2.xaxis.set_ticklabels(['No Crowd Dispersal', 'Crowd Dispersal']);
ax2.yaxis.set_ticklabels(['No Crowd Dispersal', 'Crowd Dispersal'], va='center');
    

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))
plt.subplots_adjust(wspace=0.5)

sns.heatmap(cm_array[5].astype(int), annot=True, ax = ax1, cmap='Reds', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax1.set_xlabel('Predicted State Response');
ax1.set_ylabel('Actual State Response'); 
ax1.set_title('Confusion Matrix'); 
ax1.xaxis.set_ticklabels(['No Killings', 'Killings']);
ax1.yaxis.set_ticklabels(['No Killings', 'Killings'], va='center');

sns.heatmap(cm_array[6].astype(int), annot=True, ax = ax2, cmap='Reds', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax2.set_xlabel('Predicted State Response');
ax2.set_ylabel('Actual State Response'); 
ax2.set_title('Confusion Matrix'); 
ax2.xaxis.set_ticklabels(['No Shootings', 'Shootings']);
ax2.yaxis.set_ticklabels(['No Shootings', 'Shootings'], va='center');
    

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(6,4))
plt.subplots_adjust(wspace=0.5)

sns.heatmap(cm_array[4].astype(int), annot=True, ax = ax1, cmap='Blues', fmt='d'); #annot=True to annotate cells

# labels, title and ticks
ax1.set_xlabel('Predicted State Response');
ax1.set_ylabel('Actual State Response'); 
ax1.set_title('Confusion Matrix'); 
ax1.xaxis.set_ticklabels(['No Ignore', 'Ignore']);
ax1.yaxis.set_ticklabels(['No Ignore', 'Ignore'], va='center');

In [ ]:
print( classification_report(y_test,y_preds))

##### Get predictions from all rows in X?

In [ ]:
y_modeled = tuned_model.predict(X)

In [ ]:
y_modeled[0:5]

#### Compare to model using ClassifierChain

In [ ]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

# using classifier chains
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_sc, y_train)
# predict
predictions = classifier.predict(X_test_sc)


In [ ]:
# accuracy
print("Accuracy = ", accuracy_score(y_test,predictions))
print("\n")